In [1]:
import rospy
import math
import sys
import select
import actionlib
from nav_msgs.msg import Odometry
from std_srvs.srv import *
from geometry_msgs.msg import Point, Pose, Twist
from ros_simulation.msg import Pos
from sensor_msgs.msg import LaserScan
from ros_simulation.msg import PlanningAction, PlanningGoal
from ros_simulation.srv import Goals

%matplotlib widget 
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import FloatText, VBox, widgets, Layout

position = None
linear_velocity = None

x_plot = []
y_plot = []

# Initialize lists to store target positions
targets_set = 0
targets_cancelled = 0

# Create a publisher to publish position information
pub = rospy.Publisher('/pos', Pos, queue_size=1)

# Create an action client for the reaching_goal
action_client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)

# Create a goal message
goal = PlanningGoal()

# Create a publisher to send the target
goal_pub = rospy.Publisher('/goal_topic', Point, queue_size=1)

In [2]:
obstacle_distance = FloatText(description='Obstacle Distance:')

def callback_scan(data):
        obstacle_distance.value = round(min(data.ranges), 2)

In [3]:
# Define callback function for target set button
def target_set_callback(sender):
    # Get x and y coordinates from text boxes
    x = float(x_textbox.value)
    y = float(y_textbox.value)
    
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    
    goal_msg = Point()
    goal_msg.x = goal.target_pose.pose.position.x
    goal_msg.y = goal.target_pose.pose.position.y
    goal_pub.publish(goal_msg)

    action_client.send_goal(goal)

In [4]:
# Define callback function for target cancel button
def target_cancel_callback(sender):
    action_client.cancel_goal()

In [5]:
# Define the get goals client to know the reached and cancelled goals
goal_reached = FloatText(description='Goal Reached:')
goal_cancelled = FloatText(description='Goal Cancelled:')

def get_goals_client():
    # Create a ROS service client for the 'goals_srv' service
    rospy.wait_for_service('goals_srv')
    try:
        # Create a proxy to call the 'goals_srv' service
        get_goals = rospy.ServiceProxy('goals_srv', Goals)
        # Make a request to the service
        response = get_goals()
        # Access the values of reached and cancelled goals from the response
        goal_reached.value = response.reached
        goal_cancelled.value = response.cancelled
    except rospy.ServiceException as e:
        goal_reached.value = goal_reached.value
        goal_cancelled.value = goal_cancelled.value

In [6]:
def plot(x_plot, y_plot):
    np_x_plot = np.array(x_plot)
    np_y_plot = np.array(y_plot)
    ax.plot(np_x_plot, np_y_plot)
    
# to plot the robot position
np_x_value = FloatText(description='X POSITION:')
np_y_value = FloatText(description='Y POSITION:')

np_x_plot = np.array(x_plot)
np_y_plot = np.array(y_plot)
fig, ax = plt.subplots()
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.plot(np_x_plot,np_y_plot)
ax.set_title("ROS&jupyter_simulation")
ax.set_xlabel("x")
ax.set_ylabel("y")

def odom_callback(data):
    # Store the position and linear velocity information from the odometry message
    position = data.pose.pose.position
    linear_velocity = data.twist.twist.linear

    # Create a new message to publish position and velocity information
    global x_plot, y_plot
    x_plot.append(data.pose.pose.position.x)
    y_plot.append(data.pose.pose.position.y)
    
    # to change te value with the last into the array
    np_x_value.value = x_plot[-1]
    np_y_value.value = y_plot[-1]
    plot(x_plot, y_plot)
    
    msg = Pos()
    msg.x = position.x
    msg.y = position.y
    msg.vx = linear_velocity.x
    msg.vy = linear_velocity.y
    
    # Publish the message
    pub.publish(msg)
    
    # Call the function to retrive the goals information
    get_goals_client()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# initialize the node
rospy.init_node("Client_jupyter")

# Create a subscriber to listen for odometry information
sub_odom = rospy.Subscriber('/odom', Odometry, odom_callback)

# Create a subscriber to listen for laser information
sub_scan = rospy.Subscriber('/scan', LaserScan, callback_scan)

# Call the function to retrive the goals information
get_goals_client()

# Create text boxes for target coordinates
x_textbox = widgets.FloatText(description='X:', layout=Layout(width='150px'))
y_textbox = widgets.FloatText(description='Y:', layout=Layout(width='150px'))

# Create buttons to set and cancel targets
target_set_button = widgets.Button(description='Set Target')
target_cancel_button = widgets.Button(description='Cancel Target')

# Attach callback functions to buttons
target_set_button.on_click(target_set_callback)
target_cancel_button.on_click(target_cancel_callback)

# Display widgets
display(VBox([np_x_value,np_y_value]))
display(widgets.HBox([x_textbox, y_textbox, target_set_button, target_cancel_button]))
display(VBox([obstacle_distance,goal_reached,goal_cancelled]))